# Impressions on Video Game Developers from Online Forums

## Reasoning:
Knowledge of webscraping gives access to the largest bank of data available. Almost any website can become a source of data. Its use can range from analyzing competitors to learning more about a user base.

It would make sense to scrape through comments of each post too, but that would take far too long. GameFaqs is at least very casual so posts there will be as opinionated as comments.


## Objective:
The slightly long-winded title explains most of what this notebook is about. My goal is to scrape the post titles from Reddit and Gamefaqs to perform sentiment analysis on them on top of a some data exploration.


## Methods:
I will be using selenium to scroll through all reddit posts and to do some other automation used for clicking buttons. BeautifulSoup will be used to scrape and retrieve the actual data.

I will need to scrape a list of current common user agents and lists of free, recent proxies to rotate through for GameFaqs. They use pages and not infinite scrolling so too many http requests will result in a ban. To play it safe and uninterrupted a few measures will be taken. These IPs and user agents could be used for scraping any other website as well.

I will be obtaining all posts from the past year and filter out the ones which don't mention any that don't mention a developer.

## Featured:
- Webscraping with BeautifulSoup and Selenium
- Advanced knowledge on how to rotate proxies and user agents
- Working with Pandas DataFrames
- Data analysis and visualization


In [77]:
import requests
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

## Scraping reddit

<br>
Here I create a function which uses a web driver to simulate scrolling and load in all results of the page (reddit limits to around 1000 posts). If I wanted to access the entire archive, there are websites which store all of Reddit's data. However, since I am also getting plenty of data from other forums there is not much need to scrape the archives for the purpose of this notebook.

After the entire page is loaded, we can scrape all the text of each post title.

In [65]:
def reddit_scraper(url):
    '''
    Webscrapes all reddit posts from the given link by scrolling through the "infinite scrolling"
    
    Args:
        url: The url of the subreddit or other reddit page you'd like to scrape
    
    Returns:
        A list of all post titles on that page
    '''
    
    driver = webdriver.Chrome(ChromeDriverManager().install())

    driver.get(url)

    for n in range(600): 
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

        time.sleep(0.5)

        
    page_html = soup(driver.page_source, 'lxml')

    driver.close()
    
    containers = page_html.findAll("a", {'data-click-id' : 'body'})

    post_titles = []
    for container in containers:

        titles = container.find_all("h2", recursive=True)

        for title_tag in titles:
            post_titles.append(title_tag.text)

            
    return post_titles

### Scraping r/games

In [280]:
r_games_url = 'https://www.reddit.com/r/games/top/?t=year'
r_games_posts = reddit_scraper(r_games_url)


Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/adrianherrmann/.wdm/chromedriver/2.46/mac64/chromedriver


### Scraping r/gaming

In [281]:
r_gaming_url = 'https://www.reddit.com/r/gaming/top/?t=year'
r_gaming_posts = reddit_scraper(r_gaming_url)


Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/adrianherrmann/.wdm/chromedriver/2.46/mac64/chromedriver


### Scraping r/truegaming

In [282]:
r_truegaming_url = 'https://www.reddit.com/r/truegaming/top/?t=year'
r_truegaming_posts = reddit_scraper(r_truegaming_url)


Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/adrianherrmann/.wdm/chromedriver/2.46/mac64/chromedriver


### Let's combine all the results from each subreddit

In [289]:
# r/games
r_games_forums = pd.concat([pd.DataFrame([[title, 'Reddit', 'r/games']], columns=['Post', 'Website', 'Board']) 
                           for title in r_games_posts], 
                           ignore_index=True)

# r/gaming
r_gaming_forums = pd.concat([pd.DataFrame([[title, 'Reddit', 'r/gaming']], columns=['Post', 'Website', 'Board']) 
                            for title in r_gaming_posts], 
                            ignore_index=True)

# r/truegaming
r_truegaming_forums = pd.concat([pd.DataFrame([[title, 'Reddit', 'r/truegaming']], columns=['Post', 'Website', 'Board']) 
                                for title in r_truegaming_posts], 
                                ignore_index=True)

# Join all to post_titles
post_titles = pd.DataFrame(columns=['Post', 'Website', 'Board'])
post_titles = post_titles.append([r_games_forums, r_gaming_forums, r_truegaming_forums], ignore_index=True)

### Progress Report
Let's check out the DataFrame so far

In [290]:
print('Shape: {}'.format(post_titles.shape))
post_titles.head(5)

Shape: (3007, 3)


Post Website    Board
0  John @Totalbiscuit Bain July 8, 1984 - May 24,...  Reddit  r/games
1                      Bungie Splits With Activision  Reddit  r/games
2  Totalbiscuit hospitalized, his cancer is sprea...  Reddit  r/games
3                           [E3 2018] Cyberpunk 2077  Reddit  r/games
4         Sony faces growing Fortnite backlash at E3  Reddit  r/games

## Scraping GameFaqs

GameFaqs will be a different challenge. Rather than infinite scrolling, this website uses pages. This means many http requests will need to be made, so in order to avoid an ip ban and not strain their servers a few things must be done:

1. Rotate user agents
2. Rotate ip addresses
3. Sleep on each request

### 1. Obtaining User Agents to Rotate Through

To accomplish this we have to make use of whatismybrowser.com's list of common current user agents, which means webscraping the page to stay up to date.

In [224]:
def get_agents(browser, num_agents=10, offset=0):
    '''
    Webscrapes whatismybrowser.com for new user agents
    
    Args:
        browser: the browser you want the agent from
        num_agents: number of agents to return
        offset: get agents starting from offset num. on page
    
    Returns:
        A list of user agents from the given browser
    '''
    
    if offset + num_agents > 50:
        return []
    
    try:
        chrome_url = requests.get('https://developers.whatismybrowser.com/useragents/explore/software_name/' \
                                  + browser)
    except:
        print('Browser does not exist. Try lower case')
        return
        

    chrome_html = soup(chrome_url.content)

    chrome_containers = chrome_html.findAll('td', {'class' : 'useragent'})

    user_agents = []
    for i in range(num_agents):

        chrome_agent = chrome_containers[i + offset].a.text

        user_agents.append(chrome_agent)
    
    return user_agents

Get 10 user agents for chrome and 10 for firefox

In [175]:
user_agents = []
user_agents.extend(get_agents('chrome'))
user_agents.extend(get_agents('firefox'))

### 2. Obtaining IP Addresses to Rotate Through

We need to create a similar function for retrieving new proxies. This function is more important to call frequently as IPs should be updated frequently.

In [93]:
def get_ips(num_addresses=20):
    '''
    Webscrapes free-proxy-list.net for new free proxies. This is important because these proxies
    could go bad after just a couple hours.
    
    Args:
        num_addresses: The number of IPs you want returned. If fewer than requested are available,
        return the available amount
    
    Returns:
        A list of new proxies
    '''
        
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get('https://free-proxy-list.net/')
    
    page_html = soup(driver.page_source, 'lxml')
    containers = page_html.findAll('tr', {'role' : 'row'})
    
    ips = []
    ip_num = 0
    page_num = 1
    next_set_btn = driver.find_element_by_xpath('//*[@id="proxylisttable_next"]/a')
    while len(ips) < num_addresses:
                    
        ip_num += 1
        
        # Click next button to get more ips if the current page doesn't have enough
        if ((ip_num % 20) - 1 == 0) and ip_num != 1:
            
            # If reached the last page, return what we have
            if page_num >= 15:
                driver.close()
                return ips
            
            next_set_btn.click()
            next_set_btn = driver.find_element_by_xpath('//*[@id="proxylisttable_next"]/a')
            
            ip_num = 1
            page_html = soup(driver.page_source, 'lxml')
            containers = page_html.findAll('tr', {'role' : 'row'})
            
            page_num += 1
        
        row = containers[ip_num].find_all('td')
        
        ip = row[0].text
        port = row[1].text
        
        if row[6].text == 'yes':
            ips.append(':'.join([ip, port]))

    driver.close()
    
    return ips

In [94]:
ips = get_ips(20)


Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/adrianherrmann/.wdm/chromedriver/2.46/mac64/chromedriver


In [95]:
# Print out the proxies to see what they look like
ips

['51.68.112.254:3128',
 '45.32.42.234:8080',
 '178.128.54.73:8080',
 '104.248.16.45:8080',
 '177.38.66.255:45235',
 '95.47.180.171:53484',
 '138.186.23.9:40340',
 '182.160.119.254:56229',
 '103.250.157.43:38641',
 '115.127.39.66:55474',
 '88.210.71.234:46626',
 '177.94.206.67:60666',
 '1.10.186.157:55129',
 '176.197.103.210:53281',
 '109.201.97.235:39125',
 '31.43.143.15:8181',
 '193.213.89.72:51024',
 '183.82.118.87:8080',
 '41.84.131.78:53281',
 '93.77.78.123:42803']

### 3. GameFaqs Scraping Function with Pauses

In [249]:
def gamefaqs_scraper(url, num_pages, ips, user_agents, offset=0, start_page=0):
    '''
    Scrape GameFaqs forums for post titles
    
    Args:
        url: The url to the first page of GameFaqs
        num_pages: Number of pages to scrape
        
    Returns:
        A list of post titles
    '''
    
    rot_list = []
    for ip in ips:
        rot_list.append([{'https' : ip}, {'User-Agent' : random.choice(user_agents)}])
        
        
    req = ''
    i = 0
    while req == '':
        try:
            agent_proxy_pair = random.choice(rot_list)
            proxy = agent_proxy_pair[0]
            headers = agent_proxy_pair[1]
            
            if start_page == 0:
                req = requests.get(url, headers=headers, proxies=proxy, timeout=10)
            else:
                req = requests.get(url + '?page=' + str(start_page+1), headers=headers, proxies=proxy, timeout=10)
                
            print('Success with IP, ' + proxy['https'])
            
            page_html = soup(req.content)

            containers = page_html.findAll('td', {'class' : 'topic'})
            
            if not containers:
                print('Agent may be banned, removing agent and trying a new one...')
                print(page_html, user_agent)
                try:
                    user_agents.remove(headers['User-Agent'])    
                except:
                    pass
                
                req = ''

        except Exception as e:
            i += 1
            print('Error with IP, ' + proxy['https'] + ' requesting a new one...')
        
            if i % 20 == 0:
                ips = get_ips(20)
                
                rot_list = []
                for ip in ips:
                    rot_list.append([{'https' : ip}, {'User-Agent' : random.choice(user_agents)}])
                 
    
    post_titles = []
    for page in range(start_page, num_pages):
        
        for container in containers:
            title = container.a.text
                 
            post_titles.append(title)
        
        
        time.sleep(3)
        
        req = ''
        i = 0
        while req == '':
            try:        
                agent_proxy_pair = random.choice(rot_list)
                proxy = agent_proxy_pair[0]
                headers = agent_proxy_pair[1]

                req = requests.get(url + '?page=' + str(page + 1), headers=headers, proxies=proxy, timeout=10)
                
                page_html = soup(req.content)

                containers = page_html.findAll('td', {'class' : 'topic'})

                if not containers:
                    print('Agent may be banned, removing agent and trying a new one...')
                    
                    try:
                        rot_list.remove(agent_proxy_pair)
                        if not rot_list:
                            print('Loading in new IPs...')
                            ips = get_ips(20)

                            rot_list = []
                            for ip in ips:
                                rot_list.append([{'https' : ip}, {'User-Agent' : random.choice(user_agents)}])

                            
                        user_agents.remove(headers['User-Agent'])    
                    except:
                        pass
                    
                    req = ''
                    time.sleep(2)
                    
                    if len(user_agents) == 0:
                        print('No more agents, ended at page,', page+1)
                        return post_titles
                else:
                    print('Success with IP ' + proxy['https'] + ', now onto page ', page + 2)
            
            except Exception as e:
                i += 1
                time.sleep(2)
                print('Error with IP ' + proxy['https'] + ', requesting a new one...')
    
                if i % 20 == 0:
                    print('Loading in new IPs...')
                    ips = get_ips(20)
                    
                    rot_list = []
                    for ip in ips:
                        rot_list.append([{'https' : ip}, {'User-Agent' : random.choice(user_agents)}])
                    
        if page % 100 == 0:
            print('Loading in new IPs...')
            ips = get_ips(20)
            
            rot_list = []
            for ip in ips:
                rot_list.append([{'https' : ip}, {'User-Agent' : random.choice(user_agents)}])
            
            
    return post_titles

### Nintendo Switch forums

In [155]:
switch_url = 'https://gamefaqs.gamespot.com/boards/189706-nintendo-switch'
switch_posts = gamefaqs_scraper(switch_url, num_pages=1700, ips=ips, user_agents=user_agents)

Success with IP, 103.103.212.222:53281
Error with IP 186.42.124.190:58607, requesting a new one...
Success with IP 1.10.186.102:52895, now onto page  2
Loading in new IPs...

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/adrianherrmann/.wdm/chromedriver/2.46/mac64/chromedriver
Success with IP 1.10.187.158:60495, now onto page  3
Error with IP 5.45.64.97:3128, requesting a new one...
Success with IP 185.68.28.76:443, now onto page  4
Success with IP 167.114.96.126:80, now onto page  5
Success with IP 62.85.62.66:53281, now onto page  6
Success with IP 190.210.230.192:40169, now onto page  7
Success with IP 186.42.124.190:58607, now onto page  8
Error with IP 5.45.64.97:3128, requesting a new one...
Success with IP 103.103.212.222:53281, now onto page  9
Success with IP 167.114.96.126:80, now onto page  10
Success with IP 27.111.38.89:44317, now onto page  11
Success with IP 196.2.14.250:43701, now onto page  12
Success with IP 167.114.96.126:80, now onto page  13


Error with IP 187.33.245.30:41396, requesting a new one...
Error with IP 187.33.245.30:41396, requesting a new one...
Success with IP 111.118.150.193:53824, now onto page  103
Success with IP 103.9.188.216:54906, now onto page  104
Success with IP 89.31.42.126:8080, now onto page  105
Success with IP 111.118.150.193:53824, now onto page  106
Error with IP 187.33.245.30:41396, requesting a new one...
Success with IP 1.10.186.229:33900, now onto page  107
Error with IP 168.194.248.156:30661, requesting a new one...
Success with IP 111.118.150.193:53824, now onto page  108
Success with IP 125.26.99.108:46339, now onto page  109
Success with IP 125.26.99.108:46339, now onto page  110
Success with IP 1.10.186.229:33900, now onto page  111
Success with IP 110.235.198.3:57660, now onto page  112
Success with IP 200.255.122.174:8080, now onto page  113
Error with IP 93.109.241.246:31869, requesting a new one...
Error with IP 168.194.248.156:30661, requesting a new one...
Error with IP 111.118.

Error with IP 187.33.245.30:41396, requesting a new one...
Success with IP 190.90.143.245:44127, now onto page  172
Error with IP 103.9.188.216:54906, requesting a new one...
Success with IP 202.57.49.146:58138, now onto page  173
Success with IP 1.10.186.229:33900, now onto page  174
Success with IP 111.118.150.193:53824, now onto page  175
Success with IP 200.255.122.174:8080, now onto page  176
Success with IP 190.90.143.245:44127, now onto page  177
Error with IP 5.2.200.145:44508, requesting a new one...
Success with IP 110.235.198.3:57660, now onto page  178
Success with IP 111.118.150.193:53824, now onto page  179
Success with IP 200.255.122.174:8080, now onto page  180
Success with IP 202.57.49.146:58138, now onto page  181
Error with IP 168.194.248.156:30661, requesting a new one...
Error with IP 93.109.241.246:31869, requesting a new one...
Success with IP 125.26.99.108:46339, now onto page  182
Error with IP 176.196.254.206:41491, requesting a new one...
Success with IP 125.

Error with IP 193.68.17.46:31049, requesting a new one...
Success with IP 51.68.112.254:3128, now onto page  221
Error with IP 5.2.200.145:44508, requesting a new one...
Error with IP 118.174.220.130:41464, requesting a new one...
Error with IP 177.70.64.115:30520, requesting a new one...
Error with IP 193.68.17.46:31049, requesting a new one...
Success with IP 200.255.122.174:8080, now onto page  222
Error with IP 201.234.253.24:47117, requesting a new one...
Success with IP 54.36.117.30:3128, now onto page  223
Error with IP 168.194.248.156:30661, requesting a new one...
Error with IP 93.109.241.246:31869, requesting a new one...
Error with IP 201.234.253.24:47117, requesting a new one...
Error with IP 93.109.241.246:31869, requesting a new one...
Error with IP 118.174.220.130:41464, requesting a new one...
Error with IP 5.2.200.145:44508, requesting a new one...
Success with IP 54.36.117.30:3128, now onto page  224
Success with IP 118.174.233.7:35069, now onto page  225
Error with I

Error with IP 36.90.150.252:47252, requesting a new one...
Error with IP 202.138.229.73:52507, requesting a new one...
Error with IP 168.194.248.156:30661, requesting a new one...
Error with IP 157.230.45.54:3128, requesting a new one...
Error with IP 168.194.248.156:30661, requesting a new one...
Error with IP 73.2.223.45:51155, requesting a new one...
Success with IP 51.68.112.254:3128, now onto page  248
Error with IP 181.143.193.2:46844, requesting a new one...
Error with IP 177.70.64.115:30520, requesting a new one...
Error with IP 181.143.193.2:46844, requesting a new one...
Error with IP 168.194.248.156:30661, requesting a new one...
Error with IP 168.194.248.156:30661, requesting a new one...
Error with IP 73.2.223.45:51155, requesting a new one...
Error with IP 168.194.248.156:30661, requesting a new one...
Error with IP 95.161.146.86:59147, requesting a new one...
Error with IP 73.2.223.45:51155, requesting a new one...
Error with IP 181.143.193.2:46844, requesting a new one.

...

Success with IP 116.0.2.94:33885, now onto page  1143
Success with IP 117.242.147.73:61453, now onto page  1144
Success with IP 116.0.2.94:33885, now onto page  1145
Success with IP 187.63.216.90:45966, now onto page  1146
Success with IP 88.87.136.137:32695, now onto page  1147
Success with IP 124.81.99.30:3128, now onto page  1148
Success with IP 178.19.180.203:54995, now onto page  1149
Success with IP 124.81.99.30:3128, now onto page  1150
Success with IP 185.112.249.186:3128, now onto page  1151
Success with IP 5.32.132.8:58214, now onto page  1152
Success with IP 212.24.147.62:40960, now onto page  1153
Success with IP 79.134.4.188:46559, now onto page  1154
Success with IP 190.215.118.170:35186, now onto page  1155
Success with IP 200.24.166.3:60590, now onto page  1156
Success with IP 110.74.221.169:59124, now onto page  1157
Error with IP 110.74.221.169:59124, requesting a new one...
Success with IP 124.81.99.30:3128, now onto page  1158
Success with IP 124.81.99.30:3128, now 

Success with IP 116.0.2.94:33885, now onto page  1271
Success with IP 212.24.147.62:40960, now onto page  1272
Success with IP 103.5.125.18:40211, now onto page  1273
Success with IP 187.63.216.90:45966, now onto page  1274
Success with IP 117.242.147.73:61453, now onto page  1275
Success with IP 200.146.212.209:48706, now onto page  1276
Success with IP 103.5.125.18:40211, now onto page  1277
Success with IP 116.0.2.94:33885, now onto page  1278
Success with IP 88.87.136.137:32695, now onto page  1279
Success with IP 79.134.4.188:46559, now onto page  1280
Success with IP 110.74.221.169:59124, now onto page  1281
Success with IP 187.63.216.90:45966, now onto page  1282
Success with IP 1.20.97.119:44020, now onto page  1283
Success with IP 125.99.100.61:40390, now onto page  1284
Error with IP 45.251.228.122:56383, requesting a new one...
Success with IP 1.4.157.35:46944, now onto page  1285
Success with IP 185.112.249.186:3128, now onto page  1286
Success with IP 88.87.136.137:32695, 

Error with IP 95.161.157.227:43170, requesting a new one...
Error with IP 190.9.59.198:53281, requesting a new one...
Success with IP 46.252.32.242:41125, now onto page  1367
Success with IP 190.9.59.198:53281, now onto page  1368
Error with IP 180.250.3.220:48349, requesting a new one...
Error with IP 189.199.69.122:44011, requesting a new one...
Error with IP 92.62.236.73:23500, requesting a new one...
Success with IP 185.41.112.29:57190, now onto page  1369
Error with IP 180.250.3.220:48349, requesting a new one...
Success with IP 185.41.112.29:57190, now onto page  1370
Success with IP 46.252.32.242:41125, now onto page  1371
Error with IP 92.62.236.73:23500, requesting a new one...
Success with IP 129.205.135.171:46696, now onto page  1372
Error with IP 92.62.236.73:23500, requesting a new one...
Error with IP 193.239.100.225:46694, requesting a new one...
Error with IP 189.199.69.122:44011, requesting a new one...
Error with IP 180.250.3.220:48349, requesting a new one...
Success

Success with IP 46.151.255.201:53281, now onto page  1465
Success with IP 194.187.148.221:61522, now onto page  1466
Error with IP 176.106.186.99:57121, requesting a new one...
Success with IP 191.100.24.251:21776, now onto page  1467
Error with IP 118.173.233.80:34136, requesting a new one...
Success with IP 46.151.255.201:53281, now onto page  1468
Success with IP 212.42.113.240:43472, now onto page  1469
Success with IP 191.100.24.251:21776, now onto page  1470
Success with IP 191.100.24.251:21776, now onto page  1471
Success with IP 118.174.233.8:59682, now onto page  1472
Success with IP 43.239.205.57:46580, now onto page  1473
Success with IP 194.187.148.221:61522, now onto page  1474
Success with IP 191.100.24.251:21776, now onto page  1475
Success with IP 194.187.148.221:61522, now onto page  1476
Success with IP 46.151.255.201:53281, now onto page  1477
Success with IP 194.187.148.221:61522, now onto page  1478
Success with IP 201.16.151.196:57368, now onto page  1479
Success 

Success with IP 185.186.154.198:35762, now onto page  1586
Success with IP 118.174.233.80:32830, now onto page  1587
Success with IP 180.180.156.35:49510, now onto page  1588
Success with IP 196.41.17.62:60976, now onto page  1589
Success with IP 89.18.44.16:56745, now onto page  1590
Success with IP 31.40.179.106:34284, now onto page  1591
Success with IP 89.18.44.16:56745, now onto page  1592
Success with IP 180.180.156.35:49510, now onto page  1593
Success with IP 117.242.147.73:61453, now onto page  1594
Success with IP 182.52.238.111:45639, now onto page  1595
Success with IP 89.18.44.16:56745, now onto page  1596
Success with IP 116.203.1.177:1994, now onto page  1597
Success with IP 1.20.97.4:46965, now onto page  1598
Success with IP 196.44.66.120:41753, now onto page  1599
Success with IP 200.146.212.209:48706, now onto page  1600
Success with IP 116.203.1.177:1994, now onto page  1601
Success with IP 89.18.44.16:56745, now onto page  1602
Loading in new IPs...

Checking for m

### Playstation forums

In [225]:
# Use new agents to avoid a temporary ban
user_agents = []
user_agents.extend(get_agents('chrome/2', num_agents=50))
user_agents.extend(get_agents('firefox/2', num_agents=50))
user_agents.extend(get_agents('safari/2', num_agents=50))

In [245]:
ips = get_ips(20)
playstation_url = 'https://gamefaqs.gamespot.com/boards/691087-playstation-4'
playstation_posts = gamefaqs_scraper(playstation_url, num_pages=1935, ips=ips, user_agents=user_agents)


Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/adrianherrmann/.wdm/chromedriver/2.46/mac64/chromedriver
Success with IP, 89.200.150.35:57680
Success with IP 119.247.47.37:3128, now onto page  2
Loading in new IPs...

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/adrianherrmann/.wdm/chromedriver/2.46/mac64/chromedriver
Success with IP 212.96.167.47:50680, now onto page  3
Success with IP 109.224.57.14:41207, now onto page  4
Success with IP 81.223.122.78:44074, now onto page  5
Success with IP 109.224.57.14:41207, now onto page  6
Success with IP 194.28.222.1:31227, now onto page  7
Success with IP 119.247.47.37:3128, now onto page  8
Error with IP 195.16.46.222:53281, requesting a new one...
Success with IP 124.41.211.188:42068, now onto page  9
Agent may be banned, removing agent and trying a new one...
Success with IP 41.78.172.20:41162, now onto page  10
Success with IP 94.41.92.22:35153, now onto page  11
Success with IP 41.78.172.20:41

Success with IP 170.210.80.241:3128, now onto page  117
Success with IP 168.121.69.22:8080, now onto page  118
Error with IP 118.175.93.86:57581, requesting a new one...
Success with IP 191.101.42.79:3128, now onto page  119
Success with IP 169.0.117.136:8080, now onto page  120
Success with IP 1.20.102.162:47214, now onto page  121
Agent may be banned, removing agent and trying a new one...
Agent may be banned, removing agent and trying a new one...
Error with IP 118.175.93.86:57581, requesting a new one...
Error with IP 122.160.72.92:40610, requesting a new one...
Success with IP 94.41.92.22:35153, now onto page  122
Success with IP 179.62.15.8:42850, now onto page  123
Agent may be banned, removing agent and trying a new one...
Success with IP 169.0.117.136:8080, now onto page  124
Agent may be banned, removing agent and trying a new one...
Error with IP 118.175.93.86:57581, requesting a new one...
Success with IP 170.210.80.241:3128, now onto page  125
Success with IP 179.62.15.8:4

Success with IP 1.20.102.162:47214, now onto page  198
Error with IP 182.52.74.40:39349, requesting a new one...
Agent may be banned, removing agent and trying a new one...
Success with IP 170.210.80.241:3128, now onto page  199
Error with IP 122.160.72.92:40610, requesting a new one...
Agent may be banned, removing agent and trying a new one...
Success with IP 46.167.248.86:30572, now onto page  200
Success with IP 94.41.92.22:35153, now onto page  201
Success with IP 168.121.69.22:8080, now onto page  202
Loading in new IPs...

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/adrianherrmann/.wdm/chromedriver/2.46/mac64/chromedriver
Success with IP 103.194.192.42:40511, now onto page  203
Success with IP 103.38.5.242:55552, now onto page  204
Success with IP 1.20.102.162:47214, now onto page  205
Success with IP 45.120.119.14:43493, now onto page  206
Success with IP 179.62.15.8:42850, now onto page  207
Success with IP 118.175.244.91:41196, now onto page  208
Succ

Success with IP 118.175.244.91:41196, now onto page  302
Loading in new IPs...

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/adrianherrmann/.wdm/chromedriver/2.46/mac64/chromedriver
Error with IP 154.72.197.106:48719, requesting a new one...
Success with IP 103.38.5.242:55552, now onto page  303
Success with IP 118.175.244.91:41196, now onto page  304
Error with IP 202.79.43.81:54996, requesting a new one...
Success with IP 200.94.140.50:48663, now onto page  305
Success with IP 191.37.231.202:56016, now onto page  306
Success with IP 1.10.188.20:31614, now onto page  307
Success with IP 46.201.249.7:52671, now onto page  308
Success with IP 46.167.248.86:30572, now onto page  309
Success with IP 94.41.92.22:35153, now onto page  310
Success with IP 80.245.90.33:33220, now onto page  311
Success with IP 1.20.102.162:47214, now onto page  312
Success with IP 193.160.226.89:53186, now onto page  313
Success with IP 62.117.73.154:3128, now onto page  314
Success wi

Success with IP 103.111.54.26:49781, now onto page  415
Agent may be banned, removing agent and trying a new one...
Error with IP 41.191.204.96:38411, requesting a new one...
Success with IP 78.157.52.130:8081, now onto page  416
Success with IP 193.106.170.133:38591, now onto page  417
Success with IP 78.157.52.130:8081, now onto page  418
Success with IP 179.125.25.218:56860, now onto page  419
Success with IP 96.9.87.54:32536, now onto page  420
Agent may be banned, removing agent and trying a new one...
Agent may be banned, removing agent and trying a new one...
Success with IP 5.128.32.12:51959, now onto page  421
Error with IP 41.191.204.96:38411, requesting a new one...
Agent may be banned, removing agent and trying a new one...
Success with IP 103.111.54.26:49781, now onto page  422
Error with IP 95.78.161.152:36270, requesting a new one...
Success with IP 113.161.207.105:60626, now onto page  423
Success with IP 179.125.25.218:56860, now onto page  424
Success with IP 103.239.

...

Error with IP 118.175.93.86:57581, requesting a new one...
Success with IP 1.20.102.228:36635, now onto page  1684
Agent may be banned, removing agent and trying a new one...
Agent may be banned, removing agent and trying a new one...
Agent may be banned, removing agent and trying a new one...
Agent may be banned, removing agent and trying a new one...
Agent may be banned, removing agent and trying a new one...
Agent may be banned, removing agent and trying a new one...
Agent may be banned, removing agent and trying a new one...
Agent may be banned, removing agent and trying a new one...
Error with IP 185.68.153.147:31981, requesting a new one...
Success with IP 95.179.234.99:3128, now onto page  1685
Error with IP 75.98.119.13:57859, requesting a new one...
Agent may be banned, removing agent and trying a new one...
Error with IP 87.249.40.38:51474, requesting a new one...
Agent may be banned, removing agent and trying a new one...
Agent may be banned, removing agent and trying a new 

Success with IP 108.53.19.16:56149, now onto page  1712
Error with IP 158.140.140.73:52049, requesting a new one...
Error with IP 195.154.174.241:8118, requesting a new one...
Error with IP 200.68.230.197:38275, requesting a new one...
Success with IP 103.47.93.212:51618, now onto page  1713
Success with IP 45.231.30.201:3128, now onto page  1714
Error with IP 89.186.71.86:39161, requesting a new one...
Error with IP 89.186.71.86:39161, requesting a new one...
Error with IP 222.165.234.147:31510, requesting a new one...
Error with IP 195.154.174.241:8118, requesting a new one...
Error with IP 190.90.20.229:30118, requesting a new one...
Success with IP 45.231.30.201:3128, now onto page  1715
Error with IP 101.109.255.18:30480, requesting a new one...
Success with IP 193.169.1.171:44258, now onto page  1716
Error with IP 31.209.104.52:60174, requesting a new one...
Error with IP 101.109.255.18:30480, requesting a new one...
Success with IP 193.169.1.171:44258, now onto page  1717
Succes

Error with IP 158.140.140.73:52049, requesting a new one...
Error with IP 190.90.20.229:30118, requesting a new one...
Error with IP 158.140.140.73:52049, requesting a new one...
Success with IP 181.129.58.203:40206, now onto page  1779
Error with IP 195.154.174.241:8118, requesting a new one...
Success with IP 103.47.93.212:51618, now onto page  1780
Success with IP 45.231.30.201:3128, now onto page  1781
Error with IP 103.47.93.212:51618, requesting a new one...
Success with IP 5.8.207.160:57192, now onto page  1782
Success with IP 181.129.58.203:40206, now onto page  1783
Error with IP 103.47.93.212:51618, requesting a new one...
Error with IP 202.61.120.133:34700, requesting a new one...
Error with IP 158.140.140.73:52049, requesting a new one...
Error with IP 101.109.255.18:30480, requesting a new one...
Error with IP 31.209.104.52:60174, requesting a new one...
Error with IP 91.92.80.25:32130, requesting a new one...
Error with IP 89.186.71.86:39161, requesting a new one...
Succe

Success with IP 81.223.122.78:44074, now onto page  1841
Success with IP 212.96.167.47:50680, now onto page  1842
Success with IP 81.223.122.78:44074, now onto page  1843
Success with IP 176.98.95.247:31955, now onto page  1844
Success with IP 89.200.150.35:57680, now onto page  1845
Success with IP 89.200.150.35:57680, now onto page  1846
Success with IP 139.162.1.73:3128, now onto page  1847
Success with IP 212.96.167.47:50680, now onto page  1848
Success with IP 176.98.95.247:31955, now onto page  1849
Success with IP 176.98.95.247:31955, now onto page  1850
Success with IP 195.138.94.21:54319, now onto page  1851
Error with IP 75.98.119.13:57859, requesting a new one...
Success with IP 89.200.150.35:57680, now onto page  1852
Success with IP 46.201.249.7:52671, now onto page  1853
Error with IP 87.249.40.38:51474, requesting a new one...
Error with IP 103.194.192.29:49202, requesting a new one...
Error with IP 87.249.40.38:51474, requesting a new one...
Success with IP 81.223.122.7

Success with IP 1.20.101.249:37829, now onto page  1904
Agent may be banned, removing agent and trying a new one...
Agent may be banned, removing agent and trying a new one...
Error with IP 45.6.100.250:48214, requesting a new one...
Success with IP 111.91.225.2:8080, now onto page  1905
Agent may be banned, removing agent and trying a new one...
Error with IP 75.98.119.13:57859, requesting a new one...
Success with IP 1.20.101.249:37829, now onto page  1906
Success with IP 119.192.179.46:55012, now onto page  1907
Success with IP 119.192.179.46:55012, now onto page  1908
Error with IP 75.98.119.13:57859, requesting a new one...
Error with IP 103.194.192.29:49202, requesting a new one...
Success with IP 119.192.179.46:55012, now onto page  1909
Success with IP 87.228.6.210:53281, now onto page  1910
Error with IP 75.98.119.13:57859, requesting a new one...
Error with IP 118.175.93.86:57581, requesting a new one...
Error with IP 75.98.119.13:57859, requesting a new one...
Error with IP 

### PC forums

In [250]:
# Use new agents to avoid a temporary ban
user_agents = []
user_agents.extend(get_agents('chrome/3', num_agents=50))
user_agents.extend(get_agents('firefox/3', num_agents=50))
user_agents.extend(get_agents('safari/3', num_agents=50))

In [251]:
ips = get_ips(20)
pc_url = 'https://gamefaqs.gamespot.com/boards/916373-pc'
pc_posts = gamefaqs_scraper(pc_url, num_pages=1065, ips=ips, user_agents=user_agents)


Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/adrianherrmann/.wdm/chromedriver/2.46/mac64/chromedriver
Error with IP, 103.210.120.42:44700 requesting a new one...
Success with IP, 180.211.163.194:34554
Agent may be banned, removing agent and trying a new one...
Success with IP 41.160.6.186:42506, now onto page  2
Loading in new IPs...

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/adrianherrmann/.wdm/chromedriver/2.46/mac64/chromedriver
Success with IP 103.210.120.42:44700, now onto page  3
Success with IP 180.211.163.194:34554, now onto page  4
Success with IP 195.211.173.143:53281, now onto page  5
Success with IP 41.160.6.186:42506, now onto page  6
Success with IP 180.211.163.194:34554, now onto page  7
Agent may be banned, removing agent and trying a new one...
Success with IP 115.124.85.220:60519, now onto page  8
Success with IP 119.15.91.189:42899, now onto page  9
Success with IP 188.246.106.182:42709, now onto page  10
Success wi

Error with IP 178.77.234.50:30392, requesting a new one...
Success with IP 81.223.122.78:44074, now onto page  117
Agent may be banned, removing agent and trying a new one...
Success with IP 176.98.95.247:31955, now onto page  118
Success with IP 41.78.172.20:41162, now onto page  119
Error with IP 188.136.190.125:49816, requesting a new one...
Success with IP 81.16.9.2:42726, now onto page  120
Success with IP 124.41.211.188:42068, now onto page  121
Success with IP 125.27.251.241:41528, now onto page  122
Success with IP 125.27.251.241:41528, now onto page  123
Agent may be banned, removing agent and trying a new one...
Error with IP 178.77.234.50:30392, requesting a new one...
Success with IP 186.148.172.44:40346, now onto page  124
Error with IP 178.77.234.50:30392, requesting a new one...
Success with IP 159.224.220.63:44299, now onto page  125
Success with IP 95.154.177.181:53089, now onto page  126
Success with IP 95.154.177.181:53089, now onto page  127
Error with IP 178.77.234

Success with IP 46.0.102.106:53281, now onto page  235
Success with IP 46.0.102.106:53281, now onto page  236
Error with IP 178.77.234.50:30392, requesting a new one...
Error with IP 178.77.234.50:30392, requesting a new one...
Success with IP 182.52.51.10:37198, now onto page  237
Success with IP 95.154.177.181:53089, now onto page  238
Success with IP 41.78.172.20:41162, now onto page  239
Success with IP 91.193.252.250:45385, now onto page  240
Success with IP 41.78.172.20:41162, now onto page  241
Error with IP 183.88.213.25:36729, requesting a new one...
Success with IP 91.193.252.250:45385, now onto page  242
Success with IP 169.255.136.14:45743, now onto page  243
Success with IP 41.160.6.186:42506, now onto page  244
Success with IP 95.154.177.181:53089, now onto page  245
Success with IP 139.162.1.73:3128, now onto page  246
Success with IP 139.162.1.73:3128, now onto page  247
Success with IP 169.255.136.14:45743, now onto page  248
Success with IP 169.255.136.14:45743, now o

Error with IP 191.98.198.45:55284, requesting a new one...
Error with IP 178.77.234.50:30392, requesting a new one...
Success with IP 91.193.252.250:45385, now onto page  325
Error with IP 191.98.198.45:55284, requesting a new one...
Error with IP 178.77.234.50:30392, requesting a new one...
Success with IP 46.0.102.106:53281, now onto page  326
Success with IP 125.23.43.59:80, now onto page  327
Success with IP 195.123.219.193:3128, now onto page  328
Success with IP 103.205.26.78:47917, now onto page  329
Error with IP 191.98.198.45:55284, requesting a new one...
Error with IP 125.23.43.59:80, requesting a new one...
Agent may be banned, removing agent and trying a new one...
Success with IP 95.154.177.181:53089, now onto page  330
Success with IP 195.123.219.193:3128, now onto page  331
Success with IP 176.98.95.247:31955, now onto page  332
Success with IP 41.160.6.186:42506, now onto page  333
Error with IP 36.66.235.147:47692, requesting a new one...
Error with IP 191.98.198.45:5

Success with IP 196.44.66.120:41753, now onto page  426
Success with IP 185.70.182.123:60432, now onto page  427
Error with IP 191.36.141.33:47226, requesting a new one...
Success with IP 95.140.19.9:8080, now onto page  428
Success with IP 201.251.91.118:31586, now onto page  429
Error with IP 92.244.36.68:47150, requesting a new one...
Success with IP 200.195.139.155:45382, now onto page  430
Success with IP 85.133.207.14:56728, now onto page  431
Success with IP 79.106.41.15:35876, now onto page  432
Error with IP 95.140.19.9:8080, requesting a new one...
Success with IP 77.93.34.62:40859, now onto page  433
Success with IP 185.70.182.123:60432, now onto page  434
Success with IP 103.110.88.64:48358, now onto page  435
Error with IP 191.36.141.33:47226, requesting a new one...
Success with IP 200.195.139.155:45382, now onto page  436
Success with IP 188.123.111.68:44383, now onto page  437
Success with IP 95.140.19.9:8080, now onto page  438
Error with IP 188.123.111.68:44383, reque

...

Error with IP 191.98.198.45:55284, requesting a new one...
Error with IP 191.98.198.45:55284, requesting a new one...
Error with IP 103.60.181.210:39313, requesting a new one...
Success with IP 94.236.198.160:33822, now onto page  828
Success with IP 150.242.34.218:53281, now onto page  829
Success with IP 150.242.34.218:53281, now onto page  830
Success with IP 95.161.189.26:61522, now onto page  831
Error with IP 109.226.49.151:51851, requesting a new one...
Error with IP 1.10.187.243:30870, requesting a new one...
Error with IP 217.113.17.226:39742, requesting a new one...
Error with IP 103.254.185.219:30743, requesting a new one...
Success with IP 94.236.198.160:33822, now onto page  832
Success with IP 95.161.189.26:61522, now onto page  833
Error with IP 36.67.24.61:43561, requesting a new one...
Error with IP 182.52.74.40:39349, requesting a new one...
Success with IP 95.67.67.54:53281, now onto page  834
Success with IP 94.236.198.160:33822, now onto page  835
Error with IP 191

Success with IP 94.236.198.160:33822, now onto page  868
Error with IP 1.10.187.243:30870, requesting a new one...
Error with IP 118.174.220.116:41476, requesting a new one...
Error with IP 36.67.24.61:43561, requesting a new one...
Error with IP 103.254.185.219:30743, requesting a new one...
Error with IP 61.8.66.178:8080, requesting a new one...
Error with IP 118.174.220.116:41476, requesting a new one...
Error with IP 61.8.66.178:8080, requesting a new one...
Error with IP 1.10.187.243:30870, requesting a new one...
Success with IP 95.161.189.26:61522, now onto page  869
Error with IP 103.254.185.219:30743, requesting a new one...
Success with IP 95.67.67.54:53281, now onto page  870
Error with IP 191.98.198.45:55284, requesting a new one...
Error with IP 103.254.185.219:30743, requesting a new one...
Success with IP 94.236.198.160:33822, now onto page  871
Error with IP 191.98.198.45:55284, requesting a new one...
Error with IP 186.250.119.155:47023, requesting a new one...
Error w

Error with IP 89.24.124.83:41584, requesting a new one...
Error with IP 202.91.92.21:43576, requesting a new one...
Error with IP 191.98.198.45:55284, requesting a new one...
Error with IP 109.226.49.151:51851, requesting a new one...
Error with IP 1.10.187.243:30870, requesting a new one...
Error with IP 202.91.92.21:43576, requesting a new one...
Error with IP 150.242.34.218:53281, requesting a new one...
Error with IP 202.91.92.21:43576, requesting a new one...
Error with IP 217.113.17.226:39742, requesting a new one...
Error with IP 171.100.204.146:44502, requesting a new one...
Success with IP 94.236.198.160:33822, now onto page  894
Error with IP 202.91.92.21:43576, requesting a new one...
Error with IP 103.254.185.219:30743, requesting a new one...
Error with IP 150.242.34.218:53281, requesting a new one...
Error with IP 186.250.119.155:47023, requesting a new one...
Success with IP 94.236.198.160:33822, now onto page  895
Error with IP 89.24.124.83:41584, requesting a new one..

Success with IP 125.27.10.226:40750, now onto page  957
Success with IP 41.215.81.170:59959, now onto page  958
Error with IP 109.69.1.72:33975, requesting a new one...
Error with IP 75.98.119.13:57859, requesting a new one...
Success with IP 41.215.81.170:59959, now onto page  959
Success with IP 195.211.173.143:53281, now onto page  960
Error with IP 45.6.100.250:48214, requesting a new one...
Error with IP 103.194.192.29:49202, requesting a new one...
Error with IP 75.98.119.13:57859, requesting a new one...
Success with IP 201.132.105.70:32577, now onto page  961
Error with IP 75.98.119.13:57859, requesting a new one...
Error with IP 87.249.40.38:51474, requesting a new one...
Error with IP 101.255.64.246:32426, requesting a new one...
Success with IP 201.132.105.70:32577, now onto page  962
Error with IP 103.194.192.29:49202, requesting a new one...
Error with IP 75.98.119.13:57859, requesting a new one...
Success with IP 87.26.3.40:8080, now onto page  963
Error with IP 75.98.119

Success with IP 190.12.55.210:46078, now onto page  1033
Success with IP 41.215.81.170:59959, now onto page  1034
Error with IP 87.249.40.38:51474, requesting a new one...
Success with IP 176.98.95.247:31955, now onto page  1035
Error with IP 109.69.1.72:33975, requesting a new one...
Error with IP 103.194.192.29:49202, requesting a new one...
Success with IP 176.98.95.247:31955, now onto page  1036
Success with IP 176.98.95.247:31955, now onto page  1037
Success with IP 87.26.3.40:8080, now onto page  1038
Success with IP 190.12.55.210:46078, now onto page  1039
Error with IP 103.194.192.29:49202, requesting a new one...
Success with IP 190.12.55.210:46078, now onto page  1040
Error with IP 45.6.100.250:48214, requesting a new one...
Success with IP 176.98.95.247:31955, now onto page  1041
Success with IP 125.27.10.226:40750, now onto page  1042
Success with IP 203.205.29.106:39191, now onto page  1043
Success with IP 176.98.95.247:31955, now onto page  1044
Success with IP 201.132.10

In [172]:
len(pc_posts)

11780

### Xbox One forums

In [252]:
ips = get_ips(20)
xbox_url = 'https://gamefaqs.gamespot.com/boards/691088-xbox-one'
xbox_posts = gamefaqs_scraper(xbox_url, num_pages=710, ips=ips, user_agents=user_agents)


Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/adrianherrmann/.wdm/chromedriver/2.46/mac64/chromedriver
Error with IP, 1.20.102.70:49183 requesting a new one...
Success with IP, 195.211.173.143:53281
Error with IP 186.250.49.241:61959, requesting a new one...
Error with IP 186.250.49.241:61959, requesting a new one...
Error with IP 103.254.185.219:30743, requesting a new one...
Agent may be banned, removing agent and trying a new one...
Error with IP 171.100.204.146:44502, requesting a new one...
Agent may be banned, removing agent and trying a new one...
Error with IP 1.20.102.70:49183, requesting a new one...
Error with IP 109.69.1.72:33975, requesting a new one...
Agent may be banned, removing agent and trying a new one...
Error with IP 171.100.204.146:44502, requesting a new one...
Success with IP 87.228.6.210:53281, now onto page  2
Loading in new IPs...

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/adrianherrmann/.wdm/chromedriver/2.

Success with IP 195.211.173.143:53281, now onto page  56
Error with IP 1.10.186.15:56293, requesting a new one...
Error with IP 171.100.204.146:44502, requesting a new one...
Error with IP 103.254.185.219:30743, requesting a new one...
Error with IP 1.20.102.70:49183, requesting a new one...
Success with IP 186.46.235.82:37066, now onto page  57
Error with IP 103.254.185.219:30743, requesting a new one...
Success with IP 186.46.235.82:37066, now onto page  58
Success with IP 51.140.249.129:8080, now onto page  59
Error with IP 171.100.204.146:44502, requesting a new one...
Success with IP 195.211.173.143:53281, now onto page  60
Error with IP 178.77.234.50:30392, requesting a new one...
Error with IP 91.240.98.31:53281, requesting a new one...
Error with IP 193.30.243.58:31517, requesting a new one...
Success with IP 193.30.243.58:31517, now onto page  61
Success with IP 91.240.98.31:53281, now onto page  62
Success with IP 91.240.98.31:53281, now onto page  63
Success with IP 193.30.2

Error with IP 103.205.26.78:47917, requesting a new one...
Success with IP 176.98.95.247:31955, now onto page  150
Success with IP 37.145.33.107:57732, now onto page  151
Success with IP 31.202.102.28:53281, now onto page  152
Success with IP 176.98.95.247:31955, now onto page  153
Error with IP 178.77.234.50:30392, requesting a new one...
Success with IP 37.60.35.4:37392, now onto page  154
Error with IP 159.16.106.110:80, requesting a new one...
Success with IP 37.145.33.107:57732, now onto page  155
Error with IP 178.77.234.50:30392, requesting a new one...
Success with IP 103.205.26.78:47917, now onto page  156
Success with IP 46.35.240.86:53370, now onto page  157
Success with IP 176.98.95.247:31955, now onto page  158
Success with IP 103.205.26.78:47917, now onto page  159
Success with IP 94.74.83.230:53281, now onto page  160
Success with IP 85.30.48.222:30228, now onto page  161
Success with IP 46.35.240.86:53370, now onto page  162
Success with IP 91.240.98.31:53281, now onto 

Error with IP 217.128.28.224:3128, requesting a new one...
Error with IP 185.3.202.47:59058, requesting a new one...
Success with IP 184.149.34.86:44175, now onto page  235
Success with IP 31.41.175.83:48528, now onto page  236
Error with IP 118.172.201.2:60374, requesting a new one...
Success with IP 202.57.55.242:51631, now onto page  237
Success with IP 139.180.134.238:3128, now onto page  238
Success with IP 31.41.175.83:48528, now onto page  239
Error with IP 87.249.40.38:51474, requesting a new one...
Success with IP 118.174.220.166:33842, now onto page  240
Success with IP 202.57.55.242:51631, now onto page  241
Success with IP 139.180.134.238:3128, now onto page  242
Error with IP 217.128.28.224:3128, requesting a new one...
Success with IP 139.180.134.238:3128, now onto page  243
Success with IP 118.174.220.166:33842, now onto page  244
Error with IP 101.109.83.222:36393, requesting a new one...
Error with IP 185.3.202.47:59058, requesting a new one...
Error with IP 101.109.83

Agent may be banned, removing agent and trying a new one...
Agent may be banned, removing agent and trying a new one...
Error with IP 109.69.1.72:33975, requesting a new one...
Success with IP 195.211.173.143:53281, now onto page  303
Success with IP 203.205.29.106:39191, now onto page  304
Agent may be banned, removing agent and trying a new one...
Success with IP 201.132.105.70:32577, now onto page  305
Error with IP 103.254.185.219:30743, requesting a new one...
Success with IP 36.67.212.161:37672, now onto page  306
Success with IP 201.132.105.70:32577, now onto page  307
Error with IP 75.98.119.13:57859, requesting a new one...
Success with IP 195.211.173.143:53281, now onto page  308
Success with IP 125.27.10.226:40750, now onto page  309
Error with IP 103.194.192.29:49202, requesting a new one...
Success with IP 201.132.105.70:32577, now onto page  310
Error with IP 103.254.185.219:30743, requesting a new one...
Error with IP 103.254.185.219:30743, requesting a new one...
Succes

...

Error with IP 109.69.1.72:33975, requesting a new one...
Agent may be banned, removing agent and trying a new one...
Error with IP 118.172.201.144:57140, requesting a new one...
Agent may be banned, removing agent and trying a new one...
Error with IP 159.16.106.110:80, requesting a new one...
Error with IP 159.16.106.110:80, requesting a new one...
Agent may be banned, removing agent and trying a new one...
Agent may be banned, removing agent and trying a new one...
Error with IP 217.128.28.224:3128, requesting a new one...
Error with IP 103.194.192.29:49202, requesting a new one...
Agent may be banned, removing agent and trying a new one...
Error with IP 118.172.201.144:57140, requesting a new one...
Error with IP 118.172.201.144:57140, requesting a new one...
Error with IP 159.16.106.110:80, requesting a new one...
Error with IP 124.41.240.191:38167, requesting a new one...
Error with IP 75.98.119.13:57859, requesting a new one...
Loading in new IPs...

Checking for mac64 chromedriv

Error with IP 118.172.201.144:57140, requesting a new one...
Error with IP 217.128.28.224:3128, requesting a new one...
Error with IP 124.41.240.191:38167, requesting a new one...
Error with IP 118.172.201.144:57140, requesting a new one...
Error with IP 109.69.1.72:33975, requesting a new one...
Error with IP 75.98.119.13:57859, requesting a new one...
Error with IP 118.172.201.144:57140, requesting a new one...
Error with IP 177.223.52.242:47165, requesting a new one...
Error with IP 124.41.240.191:38167, requesting a new one...
Error with IP 118.172.201.144:57140, requesting a new one...
Error with IP 103.194.192.29:49202, requesting a new one...
Error with IP 159.16.106.110:80, requesting a new one...
Loading in new IPs...

Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/adrianherrmann/.wdm/chromedriver/2.46/mac64/chromedriver
Agent may be banned, removing agent and trying a new one...
Agent may be banned, removing agent and trying a new one...
Agent may be ban

Success with IP 80.245.114.220:61478, now onto page  643
Error with IP 75.98.119.13:57859, requesting a new one...
Error with IP 217.128.28.224:3128, requesting a new one...
Error with IP 37.235.71.31:39387, requesting a new one...
Error with IP 217.128.28.224:3128, requesting a new one...
Error with IP 124.41.240.191:38167, requesting a new one...
Error with IP 103.254.185.219:30743, requesting a new one...
Error with IP 37.235.71.31:39387, requesting a new one...
Error with IP 103.194.192.29:49202, requesting a new one...
Error with IP 217.128.28.224:3128, requesting a new one...
Error with IP 49.128.160.42:44939, requesting a new one...
Error with IP 217.128.28.224:3128, requesting a new one...
Error with IP 217.128.28.224:3128, requesting a new one...
Success with IP 84.53.238.218:55173, now onto page  644
Error with IP 217.128.28.224:3128, requesting a new one...
Error with IP 103.254.185.219:30743, requesting a new one...
Error with IP 217.128.28.224:3128, requesting a new one...

Error with IP 37.235.71.31:39387, requesting a new one...
Error with IP 49.128.160.42:44939, requesting a new one...
Success with IP 84.53.238.218:55173, now onto page  678
Success with IP 84.53.238.218:55173, now onto page  679
Error with IP 37.235.71.31:39387, requesting a new one...
Error with IP 217.128.28.224:3128, requesting a new one...
Error with IP 37.235.71.31:39387, requesting a new one...
Success with IP 201.132.105.70:32577, now onto page  680
Error with IP 103.194.192.29:49202, requesting a new one...
Error with IP 103.194.192.29:49202, requesting a new one...
Error with IP 109.69.1.72:33975, requesting a new one...
Success with IP 84.53.238.218:55173, now onto page  681
Success with IP 80.245.114.220:61478, now onto page  682
Error with IP 103.194.192.29:49202, requesting a new one...
Error with IP 109.69.1.72:33975, requesting a new one...
Error with IP 75.98.119.13:57859, requesting a new one...
Error with IP 124.41.240.191:38167, requesting a new one...
Error with IP 

Error with IP 36.89.183.77:48517, requesting a new one...
Error with IP 124.41.240.191:38167, requesting a new one...
Error with IP 5.2.200.145:44508, requesting a new one...
Error with IP 109.69.1.72:33975, requesting a new one...
Error with IP 31.209.110.159:39494, requesting a new one...
Error with IP 36.89.183.77:48517, requesting a new one...
Agent may be banned, removing agent and trying a new one...
Success with IP 109.201.142.14:3128, now onto page  708
Error with IP 5.58.167.211:59680, requesting a new one...
Agent may be banned, removing agent and trying a new one...
Error with IP 5.2.200.145:44508, requesting a new one...
Error with IP 31.209.110.159:39494, requesting a new one...
Error with IP 36.89.183.77:48517, requesting a new one...
Error with IP 5.58.167.211:59680, requesting a new one...
Error with IP 217.128.28.224:3128, requesting a new one...
Error with IP 77.120.40.54:60832, requesting a new one...
Error with IP 210.11.181.221:55331, requesting a new one...
Agent 

### Removing Duplicates and Combining All the Results

In [257]:
switch_posts = list(set(switch_posts))
playstation_posts = list(set(playstation_posts))
pc_posts = list(set(pc_posts))
xbox_posts = list(set(xbox_posts))

In [291]:
# Switch Boards
switch_forums = pd.concat([pd.DataFrame([[title, 'GameFaqs', 'Switch']], columns=['Post', 'Website', 'Board']) 
                           for title in switch_posts], 
                           ignore_index=True)

# PS4 Boards
playstation_forums = pd.concat([pd.DataFrame([[title, 'GameFaqs', 'Playstation 4']], columns=['Post', 'Website', 'Board']) 
                            for title in playstation_posts], 
                            ignore_index=True)

# Xbox One Boards
xbox_forums = pd.concat([pd.DataFrame([[title, 'GameFaqs', 'Xbox One']], columns=['Post', 'Website', 'Board']) 
                                for title in xbox_posts], 
                                ignore_index=True)

# PC Boards
pc_forums = pd.concat([pd.DataFrame([[title, 'GameFaqs', 'PC']], columns=['Post', 'Website', 'Board']) 
                                for title in pc_posts], 
                                ignore_index=True)

# Join all to post_titles
post_titles = pd.concat([post_titles, switch_forums, playstation_forums, xbox_forums, pc_forums], ignore_index=True)

We now have all the posts we want and could display the final results

In [292]:
post_titles

Post   Website    Board
0       John @Totalbiscuit Bain July 8, 1984 - May 24,...    Reddit  r/games
1                           Bungie Splits With Activision    Reddit  r/games
2       Totalbiscuit hospitalized, his cancer is sprea...    Reddit  r/games
3                                [E3 2018] Cyberpunk 2077    Reddit  r/games
4              Sony faces growing Fortnite backlash at E3    Reddit  r/games
5       John “TotalBiscuit” Bain to be inducted into E...    Reddit  r/games
6       Later today, Red Dead 2 gets a new trailer. Be...    Reddit  r/games
7          List of Video Games where you can pet the dogs    Reddit  r/games
8                   It's time video game makers unionize.    Reddit  r/games
9       Bethesda Support Leaks Fallout 76 Customer Nam...    Reddit  r/games
10      Ubisoft will now ban players for racist, homop...    Reddit  r/games
11                   Fallout 76 – Official Teaser Trailer    Reddit  r/games
12      Nintendo of America’s Reggie Fils-Aime to Reti...    Reddit  r/games
13      Obsidian's The Outer Worlds blends Firefly and...    Reddit  r/games
14      Bethesda offering 500 atoms ($5 ingame store c...    Reddit  r/games
15                         [E3 2018] The Elder Scrolls VI    Reddit  r/games
16      Giantbomb Unlikely to Review Fallout 76. Gerst...    Reddit  r/games
17      Report: The Walking Dead developer Telltale Ga...    Reddit  r/games
18               Introducing the Xbox Adaptive Controller    Reddit  r/games
19      Game dev: Linux users were only 0.1% of sales ...    Reddit  r/games
20      Sony's Stubborn Stance on Cross-Play Is Embarr...    Reddit  r/games
21      Metro dev: 'if at all all the PC players annou...    Reddit  r/games
22      Blizzard Says It Wasn't Expecting Fans To Be T...    Reddit  r/games
23      Black Ops 4 adds microtransactions, requiring ...    Reddit  r/games
24      This takes it to the next level, Are we really...    Reddit  r/games
25                   EA Cancels Open-World Star Wars Game    Reddit  r/games
26      In the crazy economy of Red Dead Online, baked...    Reddit  r/games
27      PlayStation Skipping E3 For First Time in Show...    Reddit  r/games
28                   Cyberpunk 2077 is a First-Person RPG    Reddit  r/games
29      Belgian government opens criminal investigatio...    Reddit  r/games
...                                                   ...       ...      ...
107471  Skyrim vs Kingdom Come Deliverance, which game...  GameFaqs       PC
107472           MH World PC port planned for autumn 2018  GameFaqs       PC
107473                 Best place to buy cheap Steam key?  GameFaqs       PC
107474                Looking to get my A+ certification.  GameFaqs       PC
107475                            Gears of war 4 help plz  GameFaqs       PC
107476   WD Easystore 4TB External Drive $120 At Best Buy  GameFaqs       PC
107477  With Injustice 1 and MKXL being the highest se...  GameFaqs       PC
107478  Best sandbox building game? Preferably with mu...  GameFaqs       PC
107479  Switch to a 2.4ghz connection instead of 5ghz ...  GameFaqs       PC
107480                    Over-Ear Headphones under $150?  GameFaqs       PC
107481   What are your opinions on possible FO3 remaster?  GameFaqs       PC
107482               Most enjoyable fighter on pc so far?  GameFaqs       PC
107483                           Monitor recommendations?  GameFaqs       PC
107484                  What E3 games will YOU be buying?  GameFaqs       PC
107485                 Your five most played Steam games?  GameFaqs       PC
107486        New Fire Pro Wrestling game coming to Steam  GameFaqs       PC
107487                      Oculus Go is trash right now.  GameFaqs       PC
107488             Need help getting Sonic Heroes to work  GameFaqs       PC
107489  Anyone ever bypass Rockstar Social s*** on Steam?  GameFaqs       PC
107490  So does Win10 still have that mandatory update...  GameFaqs       PC
107491  Windows 10 update broke my graphics drive

## Extracting Titles which Mention Large Game Companies

First I have to make a list of relevant developers and their different nicknames.

In [313]:
# The full name is only listed in cases like 'Activision Blizzard' together with 'Activision' and 'Blizzard'
# in order to label each post in the next step
developers = [['Tencent'], ['Rockstar'], ['Valve'], ['Sony'], ['Microsoft'], ['Nintendo'], ['Bungie'],
              ['Activision Blizzard', 'Activision', 'Activi$ion', 'Blizzard'], ['Electronic Arts', 'EA'],
              ['Bandai Namco', 'Bandai', 'Namco'], ['Ubisoft'], ['Nexon'], ['Telltale'], 
              ['Epic Games', 'Epic'], ['BioWare'], ['Naughty Dog'], ['Square Enix', 'Square'], 
              ['Bunjie'], ['Insomniac'], ['Bethesda'], ['Capcom'], ['Take-Two', 'Take Two', 'Take 2', 'Take2'], 
              ['Sega'], ['Devolver Digital', 'Devolver'], ['Konami'], ['Apple']]

In [335]:
import re

In [376]:
dev_posts = pd.DataFrame(columns=['Post', 'Website', 'Board', 'Developer'])
index = 0
post_dict = {}
for i in range(len(post_titles)):

    all_developers = []
    for dev in developers:
        for nickname in dev:
            
            # Special case for EA. Common nickname but could also be mixed with common words like "each".
            match = False
            if nickname == 'EA':                
                post_title = post_titles['Post'].loc[i]
                
                # Regex to match EA outside of other words
                if re.match(r'([^a-zA-Z]|^)EA([^a-zA-Z]|$)', post_title):
                    all_developers += [dev[0]]
                    match = True
                    
            else:
                post_title = post_titles['Post'].loc[i].lower()
                
                if nickname.lower() in post_title:
                    all_developers += [dev[0]]
                    match = True

            if match:
                if post_dict.get(dev[0]):
                    post_dict[dev[0]].append(post_titles['Post'].loc[i])
                else:
                    post_dict[dev[0]] = [post_titles['Post'].loc[i]]
                break
                
    if all_developers:            
        row = post_titles.loc[i].values.tolist() + [', '.join(all_developers)]
        dev_posts.loc[index] = row
        index += 1

In [337]:
print('Shape, {}'.format(dev_posts.shape))
dev_posts.head()

Shape, (8493, 4)


Post Website    Board  \
0                      Bungie Splits With Activision  Reddit  r/games   
1         Sony faces growing Fortnite backlash at E3  Reddit  r/games   
2  Later today, Red Dead 2 gets a new trailer. Be...  Reddit  r/games   
3  Bethesda Support Leaks Fallout 76 Customer Nam...  Reddit  r/games   
4  Ubisoft will now ban players for racist, homop...  Reddit  r/games   

                     Developer  
0  Bungie, Activision Blizzard  
1                         Sony  
2           Rockstar, Take-Two  
3                     Bethesda  
4                      Ubisoft

## Sentiment Analysis

### Public Impressions of Developers

Now we can finally analyze our data and figure out how well public opinion is in each of these developer's favor.
<br>
First we want to do a simple comparison based on sentiment, this will be a 3 step process:

1. Gather all titles associated with each developer
2. Perform sentiment analysis on each title
3. Calculate the mean of the results for each developer

In [340]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/adrianherrmann/nltk_data...


True

We want to judge sentiments based on the compound score, which is the sum of all lexicon ratings standarized to be within the range from -1 to 1

In [359]:
dev_sentiments = pd.DataFrame(columns=['Mean Sentiment', 'Developer', 
                                       'Most Negative Sentence', 'Most Positive Sentence',
                                       'Most Negative Score', 'Most Positive Score',
                                       'Number of Posts'])
index = 0
sid = SentimentIntensityAnalyzer()
for dev in developers:
    titles = dev_posts[dev_posts['Developer'].str.contains(dev[0])]
    
    if not titles.values.tolist():
        continue
    
    tot_sentiment = 0
    most_neg_sent = ''
    most_pos_sent = ''
    most_neg_score = 1
    most_pos_score = -1
    for title in titles['Post'].values:
        sentiment = sid.polarity_scores(title)['compound']   
        tot_sentiment += sentiment
        
        if sentiment < most_neg_score:
            most_neg_score = sentiment
            most_neg_sent = title
            
        if sentiment > most_pos_score:
            most_pos_score = sentiment
            most_pos_sent = title
    
    mean_sentiment = tot_sentiment / len(titles)
    
    dev_sentiments.loc[index] = [mean_sentiment, dev[0],
                                 most_neg_sent, most_pos_sent,
                                 most_neg_score, most_pos_score,
                                 len(titles)]
    index += 1
    
    
    

Now that the sentiments are analyzed we can view the important details.
<br>
#### NOTE:
It's expected that some of these posts will be wrongfully rated. For example if a game has a name with a negative word and it is mentioned within the same sentence as a developer (think Resident Evil), then the title's score will negatively affect the rating. At least for now, on a grand scheme, these analyses will average out and lean toward how they are truly perceived (given the sample size is large enough).

It is important to dive deeper so that you can apply even more specific filtering and sentiment analysis when analyzing one company, which I will be doing a bit of. The post_dict created earlier will help.

In [361]:
dev_sentiments

Mean Sentiment            Developer  \
0         0.138989              Tencent   
1        -0.043847             Rockstar   
2        -0.018581                Valve   
3         0.031590                 Sony   
4         0.054865            Microsoft   
5         0.072509             Nintendo   
6         0.009765               Bungie   
7         0.006517  Activision Blizzard   
8        -0.105926      Electronic Arts   
9         0.038390         Bandai Namco   
10        0.057659              Ubisoft   
11        0.000000                Nexon   
12       -0.043905             Telltale   
13        0.028201           Epic Games   
14       -0.061895              BioWare   
15        0.051743          Naughty Dog   
16        0.037446          Square Enix   
17        0.064921            Insomniac   
18        0.034053             Bethesda   
19        0.068994               Capcom   
20       -0.113245             Take-Two   
21        0.014805                 Sega   
22        0.016383     Devolver Digital   
23        0.020019               Konami   
24        0.036826                Apple   

                               Most Negative Sentence  \
0   Does anyone actually play the crappy F2P games...   
1   Rockstar Lies & Red Dead Online Economy Is A G...   
2   Dead before it even released? A valve game?? A...   
3   Sony's Devil May Cry has arrived. Lost Souls A...   
4           NO! BAD MICROSOFT! I'm so ashamed of you!   
5   Resident Evil, Resident Evil 0, and Resident E...   
6   Activision currently under investigation for f...   
7   Heroes of the Storm pros vent sadness, anger a...   
8                  EA Head Fired For Gross Misconduct   
9                     WTF were Namco Bandai thinking?   
10  Ubisoft will now ban players for racist, homop...   
11                           What Nexon games use NX?   
12  No wonder Telltale Games died a slow painful d...   
13  God of War Has An Epic Avengers Infinity War R...   
14    Did EA ruin Bioware or did Bioware ruin itself?   
15  Naughty Dog's lead animator explains in-depth ...   
16  kingdom hearts 3 deluxe edition....what the HE...   
17  Santa Monica, Gorilla, Insomniac, Sucker Punch...   
18  Bethesda worst dev/pub of all time,nothing was...   
19  As a Devil My Cry fan, I'm jealous of the way ...   
20  Take two/Rockstar will be VERY hard to stop wi...   
21               Damn sega!!!! Killing it this week!!   
22  Not a Hero hitting Switch Aug 2nd. 12 more Dev...   
23  Death Stranding will prove that Konami was rig...   
24  Went shopping for apple products, it's a horri...   

                               Most Positive Sentence  Most Negative Score  \
0   Should superior Chinese companies like Tencent...              -0.2960   
1   Ubisoft is a BETTER company than Rockstar! LOL...              -0.8625   
2   Artifact is so good, Kotaku writer wants to re...              -0.7041   
3                  Sony wins best Float at PRIDE 2018              -0.8689   
4   Amazing show Microsoft!! My brother even said ...              -0.9191   
5   Discovered a Nintendo office close to where I ...              -0.9349   
6   LMAO Anthem is the exact same hustle Bungie us...              -0.5859   
7   Thank you Activision for CoD Black Ops 4 Black...              -0.7783   
8   EA are an excellent company that provides chea...              -0.7717   
9   Bandai Namco proves to be the best third party...              -0.6739   
10  Ubisoft is a BETTER company than Rockstar! LOL...              -0.8225   
11                           What Nexon games use NX?               0.0000   
12  Would Telltale Be The Best Developer If They H...              -0.9118   
13  The Epic Games store is now live - giving away...              -0.7717   
14  Former Bioware legend Mike Laidlaw praises wit...              -0.8225   
15  I for one am proud Naughty Dog is displaying E...              -0.6696   
16  Did Square Enix ever state why they skipped ou...              -0.8793  

### EA and Nintendo

Now let's take a deeper look at a couple of companies with scores on two opposite ends of the spectrum, Electronic Arts and Nintendo. These two have the second worst and second best scores respectively, but they also have plenty of posts, which the developers with the worst and best scores (Take-Two, 20 posts and Tencent, 9 posts) don't have.

#### Nintendo:

In [364]:
dev_sentiments[dev_sentiments['Developer'] == 'Nintendo']

Mean Sentiment Developer  \
5        0.072509  Nintendo   

                              Most Negative Sentence  \
5  Resident Evil, Resident Evil 0, and Resident E...   

                              Most Positive Sentence  Most Negative Score  \
5  Discovered a Nintendo office close to where I ...              -0.9349   

   Most Positive Score Number of Posts  
5               0.9273            3969

In [367]:
print('Nintendo\'s most negative sentence:\n' +
      dev_sentiments[dev_sentiments['Developer'] == 'Nintendo']['Most Negative Sentence'].values[0] + '\n')
print('Nintendo\'s most positive sentence:\n' +
      dev_sentiments[dev_sentiments['Developer'] == 'Nintendo']['Most Positive Sentence'].values[0] + '\n')

Nintendo's most negative sentence:
Resident Evil, Resident Evil 0, and Resident Evil 4 coming to Nintendo Switch in 2019

Nintendo's most positive sentence:
Discovered a Nintendo office close to where I live and asked if they had any kind of tour or something. Lady told me they hadn’t but she handed me a bag full of cool souvenirs. This coin is definitely the best of all!



For Nintendo it looks like the worst post, which is in fact the most negatively rated post of all threads across all developers, is rated so because it mentions the game "Resident Evil" multiple times. This only testifies for their high overall score.
<br><br>
Nintendo being so well liked comes to no surprise. They without a doubt have the most devout following of any modern gaming company. So many people grew up on Nintendo as children and continue to play their games as adults, many even strictly stick to Nintendo.
<br><br>
Let's get the word frequencies from Nintendo posts.

In [380]:
from collections import Counter

In [414]:
neg_titles = []
for title in post_dict['Nintendo']:
    sentiment = sid.polarity_scores(title)['compound']
    
    if sentiment <= -0.5:
        neg_titles.append(title)


nintendo_words = ' '.join(neg_titles).split(' ')

neu_words=[]

for word in nintendo_words:
    sentiment = sid.polarity_scores(word)['compound']
    
    if (sentiment >= -0.4 and sentiment <= 0.4):
        neu_words.append(word.lower())

neu_freq = Counter(neu_words)

print('Most common neutral words in negative titles: ', neu_freq.most_common(50))

Most common neutral words in negative titles:  [('nintendo', 187), ('the', 71), ('to', 52), ('is', 46), ('switch', 35), ('a', 32), ('of', 29), ('and', 26), ('for', 26), ('why', 26), ('in', 19), ("nintendo's", 19), ('on', 19), ('online', 19), ('you', 19), ('it', 16), ('has', 15), ('have', 14), ('do', 14), ('that', 13), ('so', 13), ('i', 12), ('are', 12), ('what', 11), ('-', 10), ('up', 10), ('games', 10), ('with', 10), ('this', 10), ('will', 9), ('sony', 9), ('if', 9), ('console', 9), ('does', 8), ('e3', 8), ('be', 8), ('an', 7), ('about', 7), ('at', 7), ('get', 7), ('nintendo?', 7), ('game', 7), ('think', 6), ('was', 6), ('or', 6), ('would', 6), ('not', 6), ('how', 6), ('most', 6), ('did', 6)]


Going down the list we see some common words, but then notice one which should definitely not be common:
<br>
'Online'

Here are some sentences containing online in the titles with negative sentiments, there are 19 posts total.

In [416]:
index = 0
bad_count = 0
while bad_count < 10:
    if 'online' in neg_titles[index].lower():
        print(neg_titles[index])
        bad_count += 1
    index += 1

Jim Sterling: The Online system makes nintendo look weak and stupid
Nintendo Switch Paid Online Still a Disaster? - Nintendo Direct Review
So nintendo online was a scam
Everytime I finish a mission in Resident Evil a Nintendo Online message appears
Would it of killed Nintendo to add promotion SNES titles to new Online Subs?
Nintendo's paid online is bad.  FACT.
scumbag nintendo wont let me try the darksouls demo without online.
Will Nintendo Switch Online kill multiplayer lobbies?
Nintendo would be dumb to not have an online paywall TBH.
What the hell does Nintendo online even include?


From this it's obvious that one very critical complaint of Nintendo is the Online system they have in place. If there was one thing they could do to please their base, it would be to address the paywall and offer more with their online subscription (i.e. it's lackluster). We can figure this all out just based on these posts.

#### Electronic Arts:

In [363]:
dev_sentiments[dev_sentiments['Developer'] == 'Electronic Arts']

Mean Sentiment        Developer              Most Negative Sentence  \
8       -0.105926  Electronic Arts  EA Head Fired For Gross Misconduct   

                              Most Positive Sentence  Most Negative Score  \
8  EA are an excellent company that provides chea...              -0.7717   

   Most Positive Score Number of Posts  
8               0.8316             130

In [369]:
print('Electronic Arts\'s most negative sentence:\n' +
      dev_sentiments[dev_sentiments['Developer'] == 'Electronic Arts']['Most Negative Sentence'].values[0] + '\n')
print('Electronic Arts\'s most positive sentence:\n' +
      dev_sentiments[dev_sentiments['Developer'] == 'Electronic Arts']['Most Positive Sentence'].values[0] + '\n')

Electronic Arts's most negative sentence:
EA Head Fired For Gross Misconduct

Electronic Arts's most positive sentence:
EA are an excellent company that provides cheap access to a lot of great games



Unlike Nintendo, many people online really hate EA. In fact, they have the most downvoted comment of any post in Reddit history, which should be a testament to how negatively they are seen. Still, they continue to be pretty successful. Apex Legends, a new game they recently released, seems to be gaining rapid popularity. Public opinion on the way they monetize their games seems to be changing, which may be a good indicator that people will once again have a positive attitude towards the company.
<br><br>
Similarly I am going to check EA's sentiment frequency.

In [418]:
neg_titles = []
for title in post_dict['Electronic Arts']:
    sentiment = sid.polarity_scores(title)['compound']
    
    if sentiment <= -0.5:
        neg_titles.append(title)


ea_words = ' '.join(neg_titles).split(' ')

neu_words=[]

for word in ea_words:
    sentiment = sid.polarity_scores(word)['compound']
    
    if (sentiment >= -0.4 and sentiment <= 0.4):
        neu_words.append(word.lower())

neu_freq = Counter(neu_words)

print('Most common neutral words in negative titles: ', neu_freq.most_common(75))

Most common neutral words in negative titles:  [('ea', 21), ('for', 9), ('star', 6), ('game', 6), ('is', 6), ('of', 4), ('open', 4), ('world', 4), ('hiring', 3), ('under', 3), ('investigation', 3), ('to', 3), ('the', 3), ('cancels', 2), ('open-world', 2), ('an', 2), ('anthem', 2), ('video', 2), ('removed', 2), ('another', 2), ('sell', 2), ('lootboxes', 2), ('in', 2), ('belgium', 2), ('games', 2), ('are', 2), ('stocks', 2), ('by', 2), ('bfv', 2), ('sales', 2), ('people', 2), ('ea:', 1), ('youtube', 1), ("creator's", 1), ('disclosure', 1), ('not', 1), ('content', 1), ('head', 1), ('misconduct', 1), ('zelda', 1), ('botw', 1), ('mass', 1), ("effect's", 1), ('franchise', 1), ('continuing', 1), ('automatically', 1), ('loses.', 1), ('conference.', 1), ('and', 1), ('got', 1), ('downgraded', 1), ('says', 1), ('singleplayer', 1), ('god', 1), ('goty.', 1), ('massive', 1), ('blow', 1), ('&', 1), ('plummet!', 1), ('should', 1), ('space', 1), ('ip', 1), ('falling', 1), ('apart.', 1), ('we', 1), ('pl

Because EA has a smaller sample size we should look at multiple words to get some more intuition.

In [421]:
index = 0
bad_count = 0
while bad_count < 10:
    if 'star' in neg_titles[index].lower() \
    or 'anthem' in neg_titles[index].lower() \
    or 'lootboxes' in neg_titles[index].lower():
        
        print(neg_titles[index])
        bad_count += 1
    index += 1

EA Cancels Open-World Star Wars Game
EA Is Hiring For An Open-World Star Wars Game
EA: YouTube creator's Anthem video removed for disclosure failure, not content
EA is under criminal investigation for continuing to sell Lootboxes in Belgium
EA Automatically loses. Horrible Conference. And Anthem got DOWNGRADED
EA's Open World Star Wars Game Cancelled
EA Vancouver hiring for open world Star Wars game
EA is hiring people for an open world star wars game...
EA cancels open world Star Wars game
EA is under criminal investigation by the Belgium government for FIFA lootboxes


The most critical complaints are on anthem and lootboxes while the negative sentiment with Star Wars seems to be more of a disappointment that a game was cancelled given the several posts on the topic. Again, the frequency size is small because the number of posts wasn't much but we can still extract a good amount of information.

## Final Remarks

There is plenty more that can be done, like getting more data from the comments. This would give much more input and allow us to view even more opinionated posts, meaning a better consensus of how people feel about different companies. This is at least a taste of what can be done